In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
data = pd.read_csv("Churn_Modelling.csv")

In [4]:
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [5]:
# Preprocessing data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
# Encoding categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [7]:
# OHE for Geography
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder()
geo_encoder=ohe_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
ohe_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
# Combine all columns with original df
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
# save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('ohe_geo.pkl','wb') as file:
    pickle.dump(ohe_geo,file)

In [12]:
# Divide into independent and dependent

X=data.drop('Exited',axis=1)
y=data['Exited']

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

# ANN IMPLEMENTATION

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [15]:
# Building ANN model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ##HL1 connected to input
    Dense(32, activation='relu'),
    Dense(1,activation="sigmoid")
])


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [18]:
## compile the model

model.compile(optimizer=opt, loss="binary_crossentropy",metrics=["accuracy"])

In [19]:
# Set up tensorboard

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%M%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [20]:
# Set up early stopping

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)


In [21]:
# train the model
history = model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.3919 - accuracy: 0.8391 - val_loss: 0.3541 - val_accuracy: 0.8565
Epoch 2/100
250/250 [==============================] - 0s 909us/step - loss: 0.3537 - accuracy: 0.8560 - val_loss: 0.3506 - val_accuracy: 0.8580
Epoch 3/100
250/250 [==============================] - 0s 883us/step - loss: 0.3479 - accuracy: 0.8605 - val_loss: 0.3604 - val_accuracy: 0.8575
Epoch 4/100
250/250 [==============================] - 0s 892us/step - loss: 0.3452 - accuracy: 0.8605 - val_loss: 0.3431 - val_accuracy: 0.8535
Epoch 5/100
250/250 [==============================] - 0s 892us/step - loss: 0.3414 - accuracy: 0.8621 - val_loss: 0.3454 - val_accuracy: 0.8560
Epoch 6/100
250/250 [==============================] - 0s 921us/step - loss: 0.3373 - accuracy: 0.8609 - val_loss: 0.3422 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 0s 888us/step - loss: 0.3350 - accuracy: 0.8631 - val_loss: 0.3443 - val_accu

In [22]:
model.save('model.h5')

/Users/shankar/Documents/DS/ann_class/ann_venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir logs/fit